# First pass through the data

In [ ]:
import numpy as np
import pandas as pd
import tifffile
import cv2 as cv
import os
import math
import matplotlib.pyplot as plt
import glob
import gc
import tensorflow as tf
import sys
import skimage.morphology
import json


In [1]:
import cv2
import json
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# load polygon mask
json_filename = r'C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\0486052bb.json'
read_file = open(json_filename, "r") 
data = json.load(read_file)
data[0]

polys = []
for index in range(data.__len__()):
    geom = np.array(data[index]['geometry']['coordinates'])
    polys.append(geom)

shape = (18484, 13013)



In [12]:
# load polygon mask
json_filename = r'C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\0486052bb.json'
read_file = open(json_filename, "r") 
data = json.load(read_file)


In [15]:
polys = []
for index in range(len(data)):
    geom = np.array(data[index]['geometry']['coordinates'])
    polys.append(geom)

shape = (18484, 13013)

In [17]:
len(polys)

130

In [ ]:
# convert to pixel mask (method 1)
mask_1 = np.zeros(shape)
for i in range(len(polys)):
    cv2.fillPoly(mask_1, polys[i], i+1)

plt.imshow(mask_1)

mask_1

range(len(polys))

In [ ]:
mask_1 = np.zeros(shape)
for i in range(len(polys)):
    cv2.fillPoly(mask_1, polys[i], i+1)

In [ ]:
#Basic parameters 
path = r"C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney"
train_path = r"C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\\"

In [ ]:
#Whats in the folder
print('\n'.join(os.listdir(path)))

In [ ]:

anatomical_files = [os.path.basename(f) for f in glob.glob(os.path.join(path, 'train/*.json')) if "anatomical" in f]
anatomical_files = [train_path + file for file in anatomical_files]
print(f'Number of anatomical_files: {len(anatomical_files)}')
print('\n'.join(anatomical_files))

In [ ]:
with open(r'C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\0486052bb.json') as f:
    file = json.load(f)

file.shape

In [ ]:
masks_files = [os.path.basename(f) for f in glob.glob(os.path.join(path, 'train/*.json')) if "anatomical" not in f]
masks_files = [r"C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train" + file for file in masks_files]
print(f'Number of masks_files: {len(masks_files)}')
print('\n'.join(masks_files))

In [ ]:
train_files = sorted(glob.glob(os.path.join(path, 'train/*.tiff')))
print(f'Number of training images: {len(train_files)}')
print('\n'.join(train_files))

In [ ]:
test_files = sorted(glob.glob(os.path.join(path, 'test/*.tiff')))
print(f'Number of test images: {len(test_files)}')
print('\n'.join(test_files))

In [ ]:
for f in train_files:
    image = tifffile.imread(f)
    print(f'Image {f} shape: {image.shape}', flush=True)
    del image
    gc.collect()
    

In [ ]:
def rel_decoder(rle_mask, image_shape):
    split = rle_mask.split()
    start, length = split[0:][::2]
    

In [ ]:
# Utility Functions
def rle_to_image(rle_mask, image_shape):
    """
    Converts an rle string to an image represented as a numpy array.
    Reference: https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode

    :param rle_mask: string with rle mask.
    :param image_shape: (width, height) of array to return
    :return: Image as a numpy array. 1 = mask, 0 = background.
    """

    # Processing
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    image = np.zeros(image_shape[0] * image_shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        image[lo:hi] = 1

    return image.reshape(image_shape).T

In [ ]:
image = tifffile.imread(r'C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\0486052bb.tiff')
image.shape

In [ ]:
class image_exploration:
    
    def __init__(self, image_file,train_df, df_info, full_image = True):
        self.image_file = image file
        self.train_df = train_df
        self.info = df_info
        self.glob_scale = 0.25
        self.plot_full_image = full_image
        
    def get_image_id(self):
        return os.path.split(r'C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\0486052bb.tiff')
    
    def read_image(self):
        image = tifffile.imread(self.image_file)
        
        if image.shape[0] == 3:
            np.transpose(image, (1,2,0))
        
        return image, (image.shape[0]. image_shape[1])
        
        
        
    def _image_analysis(self):
        image_id = self.get_image_id(self.image_file)
        image, image_shape = self.read_image(self.image_file)
        mask = self.read_mask(image, image_shape, self.glob_scale)
        
        HR_mask = self.read_mask(image, image_shape, scale=1)
        glom_df = self.get_particles(HR_mask, scale=1)
        glom_df['Id'] = image_id
        del HR_mask
        gc.collect()
        
        info = self.info[self.info['image_file'] == f'{image_id}.tiff']
        print(f'Image ID:        {image_id:}')
        print(f'Image Size:      {info["width_pixels"].values[0]} x {info["height_pixels"].values[0]}')
        print(f'Patient No:      {info["patient_number"].values[0]}')
        print(f'Sex:             {info["sex"].values[0]}')
        print(f'Age:             {info["age"].values[0]}')
        print(f'Race:            {info["race"].values[0]}')
        print(f'Height:          {info["height_centimeters"].values[0]} cm')
        print(f'Weight:          {info["weight_kilograms"].values[0]} kg')
        print(f'BMI:             {info["bmi_kg/m^2"].values[0]} kg/m^2')
        print(f'Laterality:      {info["laterality"].values[0]}')
        print(f'Percent Cortex:  {info["percent_cortex"].values[0]} %')
        print(f'Percent Medulla: {info["percent_medulla"].values[0]} %')
        
        
        

In [ ]:
train_df[train_df['id']]

In [ ]:
os.path.splitext(os.path.split(r'C:\Users\victo\Desktop\Hackaton\HuBMAP_Hacking_the_Kidney\train\0486052bb.tiff')[1])[0]

In [ ]:
train_df = pd.read_csv(os.path.join(path,'train.csv'))

rle_mask = train_df['encoding'][0]